In [1]:
import json
import psycopg2
import pandas as pd
import datetime

In [2]:
water = pd.read_csv('watersucia.csv', sep=';')

In [3]:
def read_db_config(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def create_database(config):
    connection = None
    try:
        connection = psycopg2.connect(
            host=config['host'],
            user=config['user'],
            password=config['password']
        )
        connection.autocommit = True
        cursor = connection.cursor()

        cursor.execute("SELECT 1 FROM pg_database WHERE datname='db_water';")
        if cursor.fetchone():
            print("La base de datos db_water ya existe.")
        else:
            cursor.execute("CREATE DATABASE db_water;")
            print("Base de datos db_water creada exitosamente!")

        cursor.close()
    except psycopg2.Error as e:
        print(f"No se puede crear la base de datos: {e}")
    finally:
        if connection:
            connection.close()

if __name__ == "__main__":
    config = read_db_config('db_config.json')
    create_database(config)


Base de datos db_water creada exitosamente!


In [4]:
def create_table(config, db_name):
    connection = None
    try:

        config['database'] = db_name
        connection = psycopg2.connect(**config)
        cursor = connection.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS water_table (
                "Año" DATE,
                "NombreDepartamento" VARCHAR(255),
                "Div_dpto" INT,
                "NombreMunicipio" VARCHAR(255),
                "Divi_muni" INT,
                "IrcaMinimo" FLOAT,
                "IrcaMaximo" FLOAT,
                "IrcaPromedio" FLOAT,
                "NombreParametroAnalisis2" VARCHAR(255),
                "MuestrasEvaluadas" INT,
                "MuestrasTratadas" INT,
                "MuestrasSinTratar" INT,
                "NumeroParametrosMinimo" INT,
                "NumeroParametrosMaximo" INT,
                "NumeroParametrosPromedio" INT,
                "ResultadoMinimo" FLOAT,
                "ResultadoMaximo" FLOAT,
                "ResultadoPromedio" FLOAT
            );
        """)
        print("Tabla creada con éxito.")
        connection.commit()
        cursor.close()
    except psycopg2.Error as e:
        print(f"No se puede crear la tabla: {e}")
    finally:
        if connection:
            connection.close()

if __name__ == "__main__":
    config = read_db_config('db_config.json')
    db_name = 'db_water'
    create_table(config, db_name)



Tabla creada con éxito.


In [6]:
def insertar_datos(water, config, db_name):
    connection = None
    try:
        config['database'] = db_name
        connection = psycopg2.connect(**config)
        cursor = connection.cursor()
        query = """
        INSERT INTO water_table ("Año", "NombreDepartamento", "Div_dpto", "NombreMunicipio", "Divi_muni", "IrcaMinimo", "IrcaMaximo", "IrcaPromedio", "NombreParametroAnalisis2", "MuestrasEvaluadas", "MuestrasTratadas", "MuestrasSinTratar", "NumeroParametrosMinimo", "NumeroParametrosMaximo", "NumeroParametrosPromedio", "ResultadoMinimo", "ResultadoMaximo", "ResultadoPromedio")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        for index, row in water.iterrows():
            year = datetime.date(int(row["Año"]), 1, 1)
            def convert_decimal(value):
                return float(str(value).replace(',', '.')) if pd.notnull(value) else None
            
            data = (
                year,
                row["NombreDepartamento"],
                row["Div_dpto"],
                row["NombreMunicipio"],
                row["Divi_muni"],
                convert_decimal(row["IrcaMinimo"]),
                convert_decimal(row["IrcaMaximo"]),
                convert_decimal(row["IrcaPromedio"]),
                row["NombreParametroAnalisis2"],
                row["MuestrasEvaluadas"],
                row["MuestrasTratadas"],
                row["MuestrasSinTratar"],
                row["NumeroParametrosMinimo"],
                row["NumeroParametrosMaximo"],
                row["NumeroParametrosPromedio"],
                convert_decimal(row["ResultadoMinimo"]),
                convert_decimal(row["ResultadoMaximo"]),
                convert_decimal(row["ResultadoPromedio"])
            )
            
            cursor.execute(query, data)
        connection.commit()
        print("Datos insertados exitosamente")
    except psycopg2.Error as e:
        print(f"No se puede insertar los datos: {e}")
    finally:
        if connection:
            connection.close()

if __name__ == "__main__":
    config = read_db_config('db_config.json')
    db_name = 'db_water'
    insertar_datos(water, config, db_name)
  

Datos insertados exitosamente


: 